# 실험 설계를 위한 데이터 분할

In [1]:
import os
import shutil

original_dataset_dir = './dataset'
# listdir()메서드는 해당 경로 하위에 있는 모든 폴더의 목록을 가져오는 메서드
# 이번 경우에는 폴더 이름이 클래스의 이름에 해당됨
classes_list = os.listdir(original_dataset_dir)

# 나눈 데이터를 저장할 폴더를 생성
base_dir = './splitted'
os.mkdir(base_dir)

# 분리 후에 각 데이터를 저장할 하위 폴더들 생성
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# train, val, test 하위에 각각 클래스 목록을 생성
for clss in classes_list:
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(validation_dir, clss))
    os.mkdir(os.path.join(test_dir, clss))

# 데이터 분할과 클래스별 데이터 수 확인

In [2]:
import math

for clss in classes_list:
    path = os.path.join(original_dataset_dir, clss)
    # path위치에 존재하는 모든 이미지 파일의 목록을 변수 fnames에 저장
    fnames = os.listdir(path)
    
    # Train, Validation, Test 데이터 비율을 지정
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    # Train 데이터에 해당하는 파일의 이름을 train_fnames에 저장
    train_fnames = fnames[:train_size]
    print('Train size(',clss,'): ', len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname) # 복사할 원본 파일의 경로를 지정
        dst = os.path.join(os.path.join(train_dir, clss), fname) # 복사한 후 저장할 파일의 경로를 지정
        shutil.copyfile(src, dst) # src의 경로에 해당하는 파일을 dst의 경로에 저장
        
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print('Validation size(',clss,'): ', len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, clss), fname)
        shutil.copyfile(src, dst)
    
    test_fnames = fnames[(train_size + validation_size):(validation_size + train_size + test_size)]
    print('Test size(',clss,'): ', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, clss), fname)
        shutil.copyfile(src, dst)

Train size( Apple___Apple_scab ):  378
Validation size( Apple___Apple_scab ):  126
Test size( Apple___Apple_scab ):  126
Train size( Apple___Black_rot ):  372
Validation size( Apple___Black_rot ):  124
Test size( Apple___Black_rot ):  124
Train size( Apple___Cedar_apple_rust ):  165
Validation size( Apple___Cedar_apple_rust ):  55
Test size( Apple___Cedar_apple_rust ):  55
Train size( Apple___healthy ):  987
Validation size( Apple___healthy ):  329
Test size( Apple___healthy ):  329
Train size( Cherry___healthy ):  512
Validation size( Cherry___healthy ):  170
Test size( Cherry___healthy ):  170
Train size( Cherry___Powdery_mildew ):  631
Validation size( Cherry___Powdery_mildew ):  210
Test size( Cherry___Powdery_mildew ):  210
Train size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  307
Validation size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  102
Test size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  102
Train size( Corn___Common_rust ):  715
Validation size( Corn___Commo

# 베이스라인 모델 학습을 위한 준비

In [2]:
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

BATCH_SIZE = 256
EPOCH = 5

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# transforms.Compose()는 이미지 전처리, Augmentataion등의 과정에서 사용되는 메서드
# transforms.ToTensor()은 이미지를 Tensor 형태로 변환하고, 모든 값을 0~1 사이로 정규화 함
transform_base = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])

# ImageFolder 메서드는 데이터셋을 불러오는 메서드. 지금 사용하는 이미지 데이터는 하나의 클래스가 하나의 폴더에 대응됨.
# 이때, ImageFolder 메서드를 사용. transform은 이미지를 불러온 후 전처리, 또는 Augmentation을 할 방법으 지정
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base)
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)

# DataLoader은 불러온 이미지 데이터를 주어진 조건에 따라 미니 배치 단위로 분리하는 역할을 수행
from torch.utils.data import DataLoader

# num_workers는 데이터 프로세싱에 할당하려는 CPU코어 개수(적당한 값 설정이 필요함)
# 코어 개수의 절반정도 수치면 무난하게 시스템 리소스를 사용하며 학습이 가능하다 함
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=3)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=3)

# 베이스라인 모델 설계

In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 딥러닝 모델과 관련된 기본적인 함수를 포함하는 nn.Module 클래스를 상속
class Net(nn.Module):
    
    # __init__ 함수에서 모델에서 사용할 모든 Layer를 정의
    def __init__(self):
        super(Net, self).__init__()
        
        # Conv2d(입력 채널 수, 출력 채널 수, 커널 크기)
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        # -> (64(입력 데이터의 크기) + 2 * padding(1) - 커널 크기(3)) / stride(1) + 1 = 출력 데이터의 크기(64)
        self.pool = nn.MaxPool2d(2, 2)
        # pooling 사이즈와 Stride는 일반적으로 같은 크기!
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 33)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096) # Flatten 역할
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1) # 각 클래스에 속할 확률을 output으로 출력

model_base = Net().to(DEVICE)
# 본예제에서는 학습률을 0.001로 하였지만 필자의 컴퓨터 성능이 좋지 않기 때문에....
optimizer = optim.Adam(model_base.parameters(), lr=0.01)

# 모델 학습을 위한 함수

In [13]:
def train(model, trian_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE) # data와  target변수를 사용중인 장비에 할당
        optimizer.zero_grad() # 이전 Batch의 Gradient값이 oprimizer에 저장되어 있으므로 optimizer를 초기화
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward() # 계산한 Loss 값을 바탕으로 Back Porpagation을 통해 계산한 Gradient값을 각각 Parameter에 할당
        optimizer.step() # 할당된 Parameter의 Gradient값을 이용해 모델의 Parameter를 업데이트

# 모델 평가를 위한 함수

In [14]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0 # 미니 배치별로 Loss를 합산해서 저장할 변수인 Test_loss를 선언하고, 0으로 초기화
    correct = 0
    
    # 모델을 평가하는 단계에서 모델의 Parameter를 업데이트하지 않아야 한다
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            # 모델에 입력된 Test데이터가 33개의 클래스에 속할 각각의 확률값이 Output으로 출력, 이 중 가장 높은 값을 가진
            # 인덱스를 예측값으로 저장
            pred = output.max(1, keepdim=True)[1] # 왜 1인지? [0]value추출 [1]index 추출
            # view_as
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

# 모델 학습 실행하기

In [15]:
import time
import copy

def train_baseline(model, train_loader, val_loader, optimizer, num_epochs = 5):
    best_acc = 0.0 # 정확도가 가장 높은 모델의 정확도를 저장하는 변수
    best_model_wts = copy.deepcopy(model.state_dict()) # 정확도가 가장 높은 모델을 저장할 변수
    # state_dic(): 각 계층을 매개변수 텐서로 매핑되는 Python 사전(dict) 객체
    
    for epoch in range(1, num_epochs + 1):
        since = time.time() # 한 Epoch당 소요되는 시간을 측정하기 위해 해당 Epoch를 시작할 때의 시각을 저장
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since
        print('--------------epoch {}-------------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))   
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        # early_stopping는 validation loss가 감소하였는지 확인이 필요하며,
        # 만약 감소하였을경우 현제 모델을 checkpoint로 만든다.
    model.load_state_dict(best_model_wts)
    return model

In [16]:
base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)

torch.save(base,'baseline.pt')

--------------epoch 1-------------------
train Loss: 3.2850, Accuracy: 13.40%
val Loss: 3.2847, Accuracy: 13.41%
Completed in 2m 36s
--------------epoch 2-------------------
train Loss: 2.7345, Accuracy: 23.09%
val Loss: 2.7400, Accuracy: 22.67%
Completed in 2m 38s
--------------epoch 3-------------------
train Loss: 2.0407, Accuracy: 40.16%
val Loss: 2.0556, Accuracy: 39.19%
Completed in 2m 37s
--------------epoch 4-------------------
train Loss: 1.7631, Accuracy: 47.36%
val Loss: 1.7924, Accuracy: 46.35%
Completed in 2m 36s
--------------epoch 5-------------------
train Loss: 1.8319, Accuracy: 44.18%
val Loss: 1.8687, Accuracy: 43.40%
Completed in 2m 37s


*본 예제에서는 epoch = 30, learning rate = 0.001이지만  
컴퓨터 성능이 좋지 않은 필자는 임의로 epoch = 5, learning rate = 0.01로 조정했다  
필자는 Accuracy: 44.18% 나왔지만 원래 Accuracy > 98% 정도 나온다.*

# Transfer Learning

In [23]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomHorizontalFlip(), # 좌우 반전
        transforms.RandomVerticalFlip(), # 상하 반전
        transforms.RandomCrop(52), # 이미지의 일부를 랜덤하게 잘라내어 52*52 사이즈로 변경
        transforms.ToTensor(),
        # 이미지가 Tensor 형태로 전환된 이후에 정규화를 시행. 평균값과 표준편차 값이 필요
        transforms.Normalize([0.485, 0.456, 0.406], # RGB 채널 값에서 정규화를 적용할 평균값
                            [0.229, 0.224, 0.225]) # RGB 채널 값에서 정규화를 적용할 표준편차
    ]),
    # 학습에 사용한 Augmentation 부분을 제외한 나머지 부분을 동일하게 적용
    'val': transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [26]:
import os
from torchvision.datasets import ImageFolder

In [48]:
data_dir = './splitted'
image_datasets = {x:ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers = 3)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Pre-Trained Model 불러오기

In [33]:
from torchvision import models

In [35]:
resnet = models.resnet50(pretrained=True) # pretrained 옵션을 True로 설정하면 미리 학습된 모델의 Parameter 값을 그대로 가져옴
# resnet의 출력 채널과 이 프로젝트의 채널수가 달라 마지막 Fully Connected Layer을 수정하기 위해 채널수를 저장
num_ftrs = resnet.fc.in_features
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()

# 설정한 일부 Layer의 Parameter만을 업이트 해야 하기 때문에, requires_grad = True로 설정된 Layer의 Parameter에만 적용
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr = 0.001)

from torch.optim import lr_scheduler

# StepLR() 메서드는 Epoch에 따라 Learning Rate를 변경하는 역할
# 7 Epoch마다 0.1씩 곱해 Learning Rate를 감소 시킨다는 의미
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\choic/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
24.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

61.9%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

99.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.i

# Pre-Trained Model의 일부 Layer Freeze하기

In [36]:
ct = 0 # 해당 Layer가 몇 번째 Layer인지를 나타내는 변수 ct의 값을 0으로 초기화
for child in resnet.children():
    # children() 메서드는 모델의 자식 모듈을 반복 가능한 객체로 반환하는 메서드
    ct += 1
    if ct < 6:
        for param in child.parameters(): # child.parameter()는 각 Layer의 Parameter Tesnsor를 의미
            param.requires_grad = False

# Transfer Learning 모델 학습과 검증을 위한 함수

In [49]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())  
    best_acc = 0.0  
    
    for epoch in range(num_epochs):
        print('-------------- epoch {} ----------------'.format(epoch+1)) 
        since = time.time()                                     
        for phase in ['train', 'val']: 
            if phase == 'train': 
                model.train() 
            else:
                model.eval()     
 
            running_loss = 0.0  
            running_corrects = 0  
 
            
            for inputs, labels in dataloaders[phase]: 
                inputs = inputs.to(DEVICE)  
                labels = labels.to(DEVICE)  
                
                optimizer.zero_grad() 
                
                with torch.set_grad_enabled(phase == 'train'):  
                    outputs = model(inputs)  
                    _, preds = torch.max(outputs, 1) 
                    loss = criterion(outputs, labels)  
    
                    if phase == 'train':   
                        loss.backward()
                        optimizer.step()
 
                running_loss += loss.item() * inputs.size(0)  
                running_corrects += torch.sum(preds == labels.data)  
            if phase == 'train':  
                scheduler.step()
 
            epoch_loss = running_loss/dataset_sizes[phase]  
            epoch_acc = running_corrects.double()/dataset_sizes[phase]  
 
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc)) 
 
          
            if phase == 'val' and epoch_acc > best_acc: 
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
 
        time_elapsed = time.time() - since  
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
 
    model.load_state_dict(best_model_wts) 

    return model

# 모델 학습 실행하기

In [50]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH)

torch.save(model_resnet50, 'resnet50.pt')

-------------- epoch 1 ----------------
train Loss: 0.2076 Acc: 0.9305
val Loss: 0.2805 Acc: 0.9173
Completed in 9m 19s
-------------- epoch 2 ----------------
train Loss: 0.1662 Acc: 0.9460
val Loss: 0.2415 Acc: 0.9265
Completed in 9m 27s
-------------- epoch 3 ----------------
train Loss: 0.1351 Acc: 0.9550
val Loss: 0.1161 Acc: 0.9598
Completed in 9m 26s
-------------- epoch 4 ----------------
train Loss: 0.0961 Acc: 0.9673
val Loss: 0.1623 Acc: 0.9486
Completed in 9m 26s
-------------- epoch 5 ----------------
train Loss: 0.0938 Acc: 0.9691
val Loss: 0.1353 Acc: 0.9588
Completed in 9m 27s
Best val Acc: 0.959820


# 모델평가  
+ 베이스라이 모델 평가를 위한 전처리

In [51]:
transform_base = transforms.Compose([transforms.Resize([64,64]), transforms.ToTensor()])
test_base = ImageFolder(root='./splitted/test', transform=transform_base)
test_loader_base = torch.utils.data.DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=3)

+ Transfer Leaning모델 평가를 위한 전처리

In [55]:
transform_resNet = transforms.Compose([
    transforms.Resize([64, 64]),
    transforms.RandomCrop(52),
    transforms.ToTensor(),
    transforms.Normalize([0.487, 0.456, 0.407], [0.229, 0.224, 0.255])
])

test_resNet = ImageFolder(root='./splitted/test', transform=transform_resNet)

test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=3)

+ 베이스라인 모델 성능 평가하기

In [58]:
baseline=torch.load('baseline.pt') 
baseline.eval()  
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('baseline test acc:  ', test_accuracy)

baseline test acc:   46.96457629240205


+ Transfer Learning 모델 성능 평가하기

In [59]:
resnet50=torch.load('resnet50.pt') 
resnet50.eval()  
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test acc:  ', test_accuracy)

ResNet test acc:   92.55225935661535
